In [2]:
%pip install -U llama-stack-client==0.3.0 pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 162.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 226.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 219.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
#example

from llama_stack_client import LlamaStackClient

LLAMASTACK_BASE_URL = "http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321"
MODEL = "llama-17b/llama-4-scout-17b-16e-w4a16"
TEMPERATURE = 0.2

SYSTEM_MESSAGE = (
    "You are a helpful type-a app migration assistant. Speak directly to users in a "
    "conversational and professional manner. CRITICAL do not share your internal thinking"
)

# From pod logs: current_state_name=search_app_catalog_normalize
USER_PROMPT = """You are an app migration assistant.

CRITICAL: Execute tool calls silently. Do not announce them.

Search hint (may be empty): "oom"
Latest user message: "oom-test-app"

Call search_app_catalog with:
- q = the Search hint if it is non-empty; otherwise use the Latest user message
- limit=50
- offset=0

After the tool returns, output a normalized candidate list (unique tuples only).
- If none, output exactly: CANDIDATES: NONE
- Otherwise output:
  CANDIDATES:
  1) app_id=<...> | app_name=<...> | namespace=<...> | source_cluster=<...> | destination_cluster=<...>
  2) ...
Max 50 candidates. Do not invent values.
"""

# From logs/config:
# - tools include KB file_search with vector_store_id vs_66507187-b021-4b8b-82ca-a2f277d3fcef
# - and MCP servers openshift/app-migration-utility/snow
# - state allowed_tools=["search_app_catalog"] so MCP tools are restricted to that tool name
TOOLS = [
    {
        "type": "mcp",
        "server_label": "app-migration-utility",
        "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp",
    }
]

client = LlamaStackClient(base_url=LLAMASTACK_BASE_URL)

resp = client.responses.create(
    model=MODEL,
    input=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_PROMPT},
    ],
    temperature=TEMPERATURE,
    tools=TOOLS,
)

print("response_id:", getattr(resp, "id", None))
print("status:", getattr(resp, "status", None))
print("model:", getattr(resp, "model", None))
print("-" * 80)
print("output_text:")
print(getattr(resp, "output_text", "") or "")
print("-" * 80)

# Extra debug: print output item types + any mcp_call summary
out = getattr(resp, "output", None) or []
print("output_items:", len(out))
for i, item in enumerate(out):
    print(f"[{i}] type={getattr(item, 'type', None)}")

print("-" * 80)
for item in out:
    if str(getattr(item, "type", "")) == "mcp_call":
        print(
            "mcp_call:",
            "server_label=", getattr(item, "server_label", None),
            "name=", getattr(item, "name", None),
            "error=", getattr(item, "error", None),
            "output_preview=", (getattr(item, "output", None) or "")[:300],
        )

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_7a490b87-d4f6-479d-9733-a28d135bc783
status: completed
model: llama-17b/llama-4-scout-17b-16e-w4a16
--------------------------------------------------------------------------------
output_text:
CANDIDATES:
1) app_id=oom-test-app | app_name=out memory app | namespace=oom-test | source_cluster=console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com | destination_cluster=console-openshift-console.apps.cluster-xyz.xyz.sandboxxyz.opentlc.com
--------------------------------------------------------------------------------
output_items: 3
[0] type=mcp_list_tools
[1] type=mcp_call
[2] type=message
--------------------------------------------------------------------------------
mcp_call: server_label= app-migration-utility name= search_app_catalog error= None output_preview= [{"namespace":"oom-test","app_name":"out memory app","source_cluster":"console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com","destination_cluster":"console-openshif

In [2]:
#prompt to gemini model


I am debugging the pod logs. Extract the exact and complete AGENT_NAME,STATE_NAME,TEMPERATURE, system message, user message, tools from the shared pod logs donot change them and write the below python code. if there are multiple requests response then write multiple calls to llama-stack with those requests

---------------
from llama_stack_client import LlamaStackClient

LLAMASTACK_BASE_URL = "http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321"
MODEL = "llama-17b/llama-4-scout-17b-16e-w4a16"

AGENT_NAME =
STATE_NAME = 
TEMPERATURE = 0.2

SYSTEM_MESSAGE = (
)

USER_PROMPT = """
"""

TOOLS = []

client = LlamaStackClient(base_url=LLAMASTACK_BASE_URL)

resp = client.responses.create(
    model=MODEL,
    input=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_PROMPT},
    ],
    temperature=TEMPERATURE,
    tools=TOOLS,
)

print("response_id:", getattr(resp, "id", None))
print("status:", getattr(resp, "status", None))
print("model:", getattr(resp, "model", None))
print("-" * 80)
print("output_text:")
print(getattr(resp, "output_text", "") or "")
print("-" * 80)

# Extra debug: print output item types + any mcp_call summary
out = getattr(resp, "output", None) or []
print("output_items:", len(out))
for i, item in enumerate(out):
    print(f"[{i}] type={getattr(item, 'type', None)}")

print("-" * 80)
for item in out:
    if str(getattr(item, "type", "")) == "mcp_call":
        print(
            "mcp_call:",
            "server_label=", getattr(item, "server_label", None),
            "name=", getattr(item, "name", None),
            "error=", getattr(item, "error", None),
            "output_preview=", (getattr(item, "output", None) or "")[:300],
        )
------------

pod logs


SyntaxError: invalid syntax (1074390221.py, line 4)

In [24]:
from llama_stack_client import LlamaStackClient

LLAMASTACK_BASE_URL = "http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321"
MODEL = "llama-17b/llama-4-scout-17b-16e-w4a16"
client = LlamaStackClient(base_url=LLAMASTACK_BASE_URL)

def call_llama_stack(agent_name, state_name, temperature, system_msg, user_prompt, tools):
    print(f"--- Calling Agent: {agent_name} | State: {state_name} ---")
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
        tools=tools,
    )
    
    print("response_id:", getattr(resp, "id", None))
    print("status:", getattr(resp, "status", None))
    print("output_text:")
    print(getattr(resp, "output_text", "") or "")
    
    out = getattr(resp, "output", None) or []
    for i, item in enumerate(out):
        if str(getattr(item, "type", "")) == "mcp_call":
            print(f"mcp_call [{i}]: {getattr(item, 'name', None)}")
    print("-" * 80)


In [25]:

# --- REQUEST 1: routing-agent (classify_user_intent) ---
call_llama_stack(
    agent_name="routing-agent",
    state_name="classify_user_intent",
    temperature=0.1,
    system_msg="You are a routing agent specializing in getting users to the correct specialist agent. Be helpful, friendly, and efficient in determining their needs.",
    user_prompt="""The user said: "I want to migrate an application"\n\nAnalyze their request and determine what they need help with:\n\n1. LAPTOP_REFRESH - User wants help with laptop refresh, replacement, new laptop, laptop upgrade, hardware refresh. Also includes requests that just say "refresh" or similar terms.\n2. EMAIL_CHANGE - User wants to update, change, or modify their email address\n3. TYPE_A_MIGRATION - User wants help migrating a virtual application (e.g., OVA/VMware) to Red Hat OpenShift Virtualization, or mentions MTV (Migration Toolkit for Virtualization), OpenShift Virtualization, KubeVirt, NFS-hosted OVA migration plan, or similar migration workflow requests.\n4. OTHER - User needs help with something else or request is unclear\n\nExamples of LAPTOP_REFRESH requests:\n- "I need a new laptop"\n- "laptop refresh"\n- "refresh"\n- "I want to refresh my laptop"\n- "hardware refresh"\n\nExamples of TYPE_A_MIGRATION requests:\n- "I need to migrate my app to OpenShift Virtualization"\n- "app migration"\n- "migration"\n- "I want to migrate my app"\n- "application migration"\n\nRespond with exactly one of: LAPTOP_REFRESH, EMAIL_CHANGE, TYPE_A_MIGRATION, or OTHER\n""",
    tools=[]
)

# --- REQUEST 2: type-a-app-migration (derive_app_search_q) ---
call_llama_stack(
    agent_name="type-a-app-migration",
    state_name="derive_app_search_q",
    temperature=0.1,
    system_msg="You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
    user_prompt="""You are a migration assistant.\nExtract the specific application name, ID, or namespace from the user message.\n\nCRITICAL: If the user message is generic (e.g., "migration", "ooo", "help me"),\noutput EXACTLY:\nQ: NONE\n\nUser message: "I want to migrate an application"\n\nOutput ONLY:\nQ: <search_substring>\n""",
    tools=[{"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]}, {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}, {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}]
)

# --- REQUEST 3: type-a-app-migration (classify_app_query_intent) ---
call_llama_stack(
    agent_name="type-a-app-migration",
    state_name="classify_app_query_intent",
    temperature=0.2,
    system_msg="You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
    user_prompt="""The user said: "oom-test-app"\n\nDetermine intent:\n- RETURN_TO_ROUTER: user wants to go back/stop/cancel\n- SWITCH_TASK: user is switching to a different IT task/topic (e.g., "laptop refresh", "email change")\n- QUERY: user is providing a search term or application identifier to look up\n- UNCLEAR: doesn't contain a usable query\n\nRespond with only: RETURN_TO_ROUTER, SWITCH_TASK, QUERY, or UNCLEAR\n""",
    tools=[{"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]}, {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}, {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}]
)

# --- REQUEST 4: type-a-app-migration (search_app_catalog_normalize) ---
call_llama_stack(
    agent_name="type-a-app-migration",
    state_name="search_app_catalog_normalize",
    temperature=0.2,
    system_msg="You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
    user_prompt="""You are an app migration assistant.\n\nCRITICAL: Execute tool calls silently. Do not announce them.\n\nSearch hint (may be empty): \nLatest user message: "oom-test-app"\n\nCall search_app_catalog with:\n- q = the Search hint if it is non-empty; otherwise use the Latest user message\n- limit=50\n- offset=0\n\nAfter the tool returns, output a normalized candidate list (unique tuples only).\n- If none, output exactly: CANDIDATES: NONE\n- Otherwise output:\n  CANDIDATES:\n  1) app_id=<...> | app_name=<...> | namespace=<...> | source_cluster=<...> | destination_cluster=<...>\n  2) ...\nMax 50 candidates. Do not invent values.\n""",
    tools=[{"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]}, {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]}, {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]}]
)

# --- REQUEST 5: type-a-app-migration (classify_discovery_confirmation) ---
call_llama_stack(
    agent_name="type-a-app-migration",
    state_name="classify_discovery_confirmation",
    temperature=0.2,
    system_msg="You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
    user_prompt="""The user was asked: "Should I proceed with the resource discovery for this application?"\nThey responded: "yes"\n\nClassify their intent:\n- YES: user wants to proceed with discovery\n- NO: user does not want to proceed\n- RETURN_TO_ROUTER: user wants to cancel/stop/return to router\n- UNCLEAR: ambiguous\n\nRespond with only: YES, NO, RETURN_TO_ROUTER, or UNCLEAR\n""",
    tools=[{"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]}, {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}, {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}]
)

--- Calling Agent: routing-agent | State: classify_user_intent ---


INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_1a691b6e-c531-4baf-96ff-034fcb606f04
status: completed
output_text:
Based on the user's request, "I want to migrate an application", I would categorize their request as:

TYPE_A_MIGRATION

The user explicitly mentions migrating an application, which aligns with the examples provided for TYPE_A_MIGRATION.
--------------------------------------------------------------------------------
--- Calling Agent: type-a-app-migration | State: derive_app_search_q ---
response_id: resp_d4f0f6e3-5676-49cd-9ede-64f05bb28461
status: completed
output_text:
Q: NONE
--------------------------------------------------------------------------------
--- Calling Agent: type-a-app-migration | State: classify_app_query_intent ---


INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_fad34e02-1ca3-447d-98a2-94458bfd8e16
status: completed
output_text:
QUERY
--------------------------------------------------------------------------------
--- Calling Agent: type-a-app-migration | State: search_app_catalog_normalize ---


INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_bb79bfad-4d0b-40f2-ad0a-375b4a255aa4
status: completed
output_text:
CANDIDATES:
1) app_id=oom-test-app | app_name=out memory app | namespace=oom-test | source_cluster=console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com | destination_cluster=console-openshift-console.apps.cluster-xyz.xyz.sandboxxyz.opentlc.com
mcp_call [2]: search_app_catalog
--------------------------------------------------------------------------------
--- Calling Agent: type-a-app-migration | State: classify_discovery_confirmation ---
response_id: resp_4d7e00f9-b65e-410d-87a9-7f719459b74a
status: completed
output_text:
YES
--------------------------------------------------------------------------------


In [23]:
from llama_stack_client import LlamaStackClient

LLAMASTACK_BASE_URL = "http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321"
MODEL = "llama-17b/llama-4-scout-17b-16e-w4a16"

requests = [
    {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "search_app_catalog_normalize",
        "TEMPERATURE": 0.2,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """You are an app migration assistant.

CRITICAL: Execute tool calls silently. Do not announce them.

Search hint (may be empty): 
Latest user message: "oom-test-app"

Call search_app_catalog with:
- q = the Search hint if it is non-empty; otherwise use the Latest user message
- limit=50
- offset=0

After the tool returns, output a normalized candidate list (unique tuples only).
- If none, output exactly: CANDIDATES: NONE
- Otherwise output:
  CANDIDATES:
  1) app_id=<...> | app_name=<...> | namespace=<...> | source_cluster=<...> | destination_cluster=<...>
  2) ...
Max 50 candidates. Do not invent values.
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]}
        ]
    }, {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "format_lookup_status (FAILED STATE)",
        "TEMPERATURE": 0.1,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """You are an app migration assistant.

CANDIDATES:
1) app_id=oom-test-app | app_name=out memory app | namespace=oom-test | source_cluster=console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com | destination_cluster=console-openshift-console.apps.cluster-xyz.xyz.sandboxxyz.opentlc.com

Your job is to present the catalog lookup result to the user in a conversational way.
DO NOT include internal markers like "STATUS:" in your response.

If 0 matches, respond exactly:
I couldn’t find any matching apps in the catalog. Please provide the Application Name, App ID, or Namespace.

If exactly 1 match, respond exactly in this format:
I've located <APP_NAME> (ID: <APP_ID>) in namespace <NAMESPACE>. Should I proceed with the resource discovery for this application?

If multiple matches, respond exactly in this format:
I found multiple matches for that application. Which one should we proceed with?

1) ID: <APP_ID> | Name: <APP_NAME> | Namespace: <NAMESPACE> | Cluster: <SOURCE_CLUSTER>
2) ID: ...

Please reply with the number (1 to N) or the App ID.

Rules:
- Use unique tuples only
- Show up to 15 options
- Use SOURCE_CLUSTER value for "Cluster"
- Do not invent values
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}
        ]
    }
]


client = LlamaStackClient(base_url=LLAMASTACK_BASE_URL)

for i, req in enumerate(requests):
    print(f"Executing Call {i+1}: {req['AGENT_NAME']} - {req['STATE_NAME']}")
    
    try:

        print("USER_PROMPT:", req["USER_PROMPT"])
        print("TOOLS:", req["TOOLS"])
        
        resp = client.responses.create(
            model=MODEL,
            input=[
                {"role": "system", "content": req["SYSTEM_MESSAGE"]},
                {"role": "user", "content": req["USER_PROMPT"]},
            ],
            temperature=req["TEMPERATURE"],
            tools=req["TOOLS"],
        )

        print("response_id:", getattr(resp, "id", None))
        print("status:", getattr(resp, "status", None))
        print("model:", getattr(resp, "model", None))
        print("-" * 80)
        print("output_text:")
        print(getattr(resp, "output_text", "") or "")
        print("-" * 80)

        # Extra debug: print output item types + any mcp_call summary
        out = getattr(resp, "output", None) or []
        print("output_items:", len(out))
        for j, item in enumerate(out):
            print(f"[{j}] type={getattr(item, 'type', None)}")

        print("-" * 80)
        for item in out:
            if str(getattr(item, "type", "")) == "mcp_call":
                print(
                    "mcp_call:",
                    "server_label=", getattr(item, "server_label", None),
                    "name=", getattr(item, "name", None),
                    "error=", getattr(item, "error", None),
                    "output_preview=", (getattr(item, "output", None) or "")[:300],
                )
    except Exception as e:
        print(f"Error during execution: {e}")
    
    print("=" * 100)
    print("\n")

Executing Call 1: type-a-app-migration - search_app_catalog_normalize
USER_PROMPT: You are an app migration assistant.

CRITICAL: Execute tool calls silently. Do not announce them.

Search hint (may be empty): 
Latest user message: "oom-test-app"

Call search_app_catalog with:
- q = the Search hint if it is non-empty; otherwise use the Latest user message
- limit=50
- offset=0

After the tool returns, output a normalized candidate list (unique tuples only).
- If none, output exactly: CANDIDATES: NONE
- Otherwise output:
  CANDIDATES:
  1) app_id=<...> | app_name=<...> | namespace=<...> | source_cluster=<...> | destination_cluster=<...>
  2) ...
Max 50 candidates. Do not invent values.

TOOLS: [{'type': 'file_search', 'vector_store_ids': ['vs_66507187-b021-4b8b-82ca-a2f277d3fcef']}, {'type': 'mcp', 'server_label': 'openshift', 'server_url': 'http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp', 'require_approval': 'never', 'headers': {'AUTHORITATIVE_USER_ID': 'sudash@redhat.com

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_6ce76fa2-fc0c-45c2-94d0-6b7940ee9b14
status: completed
model: llama-17b/llama-4-scout-17b-16e-w4a16
--------------------------------------------------------------------------------
output_text:
CANDIDATES:
1) app_id=oom-test-app | app_name=out memory app | namespace=oom-test | source_cluster=console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com | destination_cluster=console-openshift-console.apps.cluster-xyz.xyz.sandboxxyz.opentlc.com
--------------------------------------------------------------------------------
output_items: 4
[0] type=mcp_list_tools
[1] type=mcp_list_tools
[2] type=mcp_call
[3] type=message
--------------------------------------------------------------------------------
mcp_call: server_label= app-migration-utility name= search_app_catalog error= None output_preview= [{"namespace":"oom-test","app_name":"out memory app","source_cluster":"console-openshift-console.apps.cluster-5n8tc.5n8tc.sandbox3547.opentlc.com","destination_cl

INFO:httpx:HTTP Request: POST http://llamastack-with-config-service.llama-stack.svc.cluster.local:8321/v1/responses "HTTP/1.1 200 OK"


response_id: resp_7bff7014-5831-4862-b1bf-299fd54ffef6
status: completed
model: llama-17b/llama-4-scout-17b-16e-w4a16
--------------------------------------------------------------------------------
output_text:
I've located out memory app (ID: oom-test-app) in namespace oom-test. Should I proceed with the resource discovery for this application?
--------------------------------------------------------------------------------
output_items: 3
[0] type=mcp_list_tools
[1] type=mcp_list_tools
[2] type=message
--------------------------------------------------------------------------------




In [ ]:
# Extracted requests from pod logs
requests = [
    {
        "AGENT_NAME": "routing-agent",
        "STATE_NAME": "greet_and_identify_need",
        "TEMPERATURE": 0.3,
        "SYSTEM_MESSAGE": "You are a routing agent specializing in getting users to the correct specialist agent. Be helpful, friendly, and efficient in determining their needs.",
        "USER_PROMPT": """You are a routing agent specializing in getting users to the correct specialist agent. Be helpful, friendly, and efficient in determining their needs.

Greet the user and clearly identify yourself as the routing agent. Ask them what they need help with today.

Currently you can help with:
- Laptop refresh requests
- Email address updates
- Type-A application migration (OVA -> OpenShift Virtualization using MTV)

Ask them to describe what they need help with.
""",
        "TOOLS": []
    },
    {
        "AGENT_NAME": "routing-agent",
        "STATE_NAME": "classify_user_intent",
        "TEMPERATURE": 0.1,
        "SYSTEM_MESSAGE": "You are a routing agent specializing in getting users to the correct specialist agent. Be helpful, friendly, and efficient in determining their needs.",
        "USER_PROMPT": """The user said: "I want to migrate an application"

Analyze their request and determine what they need help with:

1. LAPTOP_REFRESH - User wants help with laptop refresh, replacement, new laptop, laptop upgrade, hardware refresh. Also includes requests that just say "refresh" or similar terms.
2. EMAIL_CHANGE - User wants to update, change, or modify their email address
3. TYPE_A_MIGRATION - User wants help migrating a virtual application (e.g., OVA/VMware) to Red Hat OpenShift Virtualization, or mentions MTV (Migration Toolkit for Virtualization), OpenShift Virtualization, KubeVirt, NFS-hosted OVA migration plan, or similar migration workflow requests.
4. OTHER - User needs help with something else or request is unclear

Examples of LAPTOP_REFRESH requests:
- "I need a new laptop"
- "laptop refresh"
- "refresh"
- "I want to refresh my laptop"
- "hardware refresh"

Examples of TYPE_A_MIGRATION requests:
- "I need to migrate my app to OpenShift Virtualization"
- "app migration"
- "migration"
- "I want to migrate my app"
- "application migration"

Respond with exactly one of: LAPTOP_REFRESH, EMAIL_CHANGE, TYPE_A_MIGRATION, or OTHER
""",
        "TOOLS": []
    },
    {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "derive_app_search_q",
        "TEMPERATURE": 0.1,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """You are a migration assistant.
Extract the specific application name, ID, or namespace from the user message.

CRITICAL: If the user message is generic (e.g., "migration", "ooo", "help me"),
output EXACTLY:
Q: NONE

User message: "I want to migrate an application"

Output ONLY:
Q: <search_substring>
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}
        ]
    },
    {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "classify_app_query_intent",
        "TEMPERATURE": 0.2,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """The user said: "oom-test-app"

Determine intent:
- RETURN_TO_ROUTER: user wants to go back/stop/cancel
- SWITCH_TASK: user is switching to a different IT task/topic (e.g., "laptop refresh", "email change")
- QUERY: user is providing a search term or application identifier to look up
- UNCLEAR: doesn't contain a usable query

Respond with only: RETURN_TO_ROUTER, SWITCH_TASK, QUERY, or UNCLEAR
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}
        ]
    },
    {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "search_app_catalog_normalize",
        "TEMPERATURE": 0.2,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """You are an app migration assistant.

CRITICAL: DO NOT announce what you are about to do. Execute tool calls silently.

Execute:
search_app_catalog(q="oom-test-app", limit=50, offset=0)

After the tool returns, output a normalized candidate list (unique tuples only).
- If none found, output exactly: CANDIDATES: NONE
- Otherwise output:
  CANDIDATES:
  1) app_id=<...> | app_name=<...> | namespace=<...> | source_cluster=<...> | destination_cluster=<...>
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": ["search_app_catalog"]}
        ]
    },
    {
        "AGENT_NAME": "type-a-app-migration",
        "STATE_NAME": "format_lookup_status (FAILED STATE)",
        "TEMPERATURE": 0.1,
        "SYSTEM_MESSAGE": "You are a helpful type-a app migration assistant. Speak directly to users in a conversational and professional manner. CRITICAL do not share your internal thinking",
        "USER_PROMPT": """You are an app migration assistant.

Candidates:
search_app_catalog(q="oom-test-app", limit=50, offset=0)

Your job is to present the catalog lookup result to the user in a conversational way.
DO NOT include internal markers like "STATUS:" in your response.

If 0 matches, respond exactly:
I couldn’t find any matching apps in the catalog. Please provide the Application Name, App ID, or Namespace.

If exactly 1 match, respond exactly in this format:
I've located <APP_NAME> (ID: <APP_ID>) in namespace <NAMESPACE>. Should I proceed with the resource discovery for this application?

If multiple matches, respond exactly in this format:
I found multiple matches for that application. Which one should we proceed with?

1) ID: <APP_ID> | Name: <APP_NAME> | Namespace: <NAMESPACE> | Cluster: <SOURCE_CLUSTER>
2) ID: ...

Please reply with the number (1 to N) or the App ID.

Rules:
- Use unique tuples only
- Show up to 15 options
- Use SOURCE_CLUSTER value for "Cluster"
- Do not invent values
""",
        "TOOLS": [
            {"type": "file_search", "vector_store_ids": ["vs_66507187-b021-4b8b-82ca-a2f277d3fcef"]},
            {"type": "mcp", "server_label": "openshift", "server_url": "http://ocp-mcp-server.llama-stack.svc.cluster.local:8000/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]},
            {"type": "mcp", "server_label": "app-migration-utility", "server_url": "http://app-migration-utility-mcp.app-migration-utility.svc.cluster.local/mcp", "require_approval": "never", "headers": {"AUTHORITATIVE_USER_ID": "sudash@redhat.com", "SERVICE_NOW_TOKEN": "now_mock_api_key"}, "allowed_tools": [""]}
        ]
    }
]